# Optimization Test

### Importing libs and data

In [2195]:
import pandas as pd
import pyomo.environ as pyo
from pathlib import Path
import os

In [2196]:
ROOT = Path(__name__).resolve().parent.parent
FILE = "generic_input_case.xlsx"
FILE_PATH = os.path.join(ROOT, "data", FILE)

In [2197]:
pdata = dict()

with pd.ExcelFile(FILE_PATH) as workbook:
    sheets = workbook.sheet_names
    for sheet in sheets:
        pdata[sheet] = pd.read_excel(workbook, sheet_name=sheet)

c:\Users\Vinicius\AppData\Local\Programs\Python\Python312\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


### Data treatment

In [2198]:
pdata["HORIZONTE"] = pdata["HORIZONTE"].drop(["CICLO_LENTO"], axis=1)
pdata["FROTA"] = pdata["FROTA"].dropna(axis=1)
pdata["BD_UP"] = pdata["BD_UP"].dropna(axis=1).drop(["DATA_COLHEITA", "IDADE_FLORESTA", "IMA", "RESERVADO"], axis=1)

In [2199]:
pdata["period_set"] = pdata["HORIZONTE"]["DIA"].unique()

pdata["logistics_set"] = pdata["FROTA"]["TRANSPORTADOR"].unique()

pdata["up_set"] = pdata["BD_UP"]["UP"].unique()
# pdata["farm_set"] = pdata["BD_UP"]["FAZENDA"].unique()
# Não há necessidade da fazenda. Cada UP é única. Posso somar tudo da UP e fazer por fazenda
# Ou posso fazer somente por UP. Nesse caso, farei por UP para tentar ser mais completo

pdata["factory_set"] = pdata["FABRICA"]["FABRICA"].unique()

In [2200]:
logistics_dict = pdata["FROTA"].set_index(["TRANSPORTADOR"]).to_dict()
pdata["min_trucks_param"] = logistics_dict["FROTA_MIN"]
pdata["max_trucks_param"] = logistics_dict["FROTA_MAX"]

grua_dict = pdata["GRUA"].set_index(["TRANSPORTADOR"]).to_dict()
pdata["grua_param"] = grua_dict["QTD_GRUAS"]

up_dict = pdata["BD_UP"].set_index(["UP"]).to_dict()
pdata["density_param"] = up_dict["DB"]
pdata["volume_param"] = up_dict["VOLUME"]
pdata["rsp_param"] = up_dict["RSP"]

factory_dict = pdata["FABRICA"].set_index(["FABRICA", "DIA",]).to_dict()
pdata["min_demand_param"] = factory_dict["DEMANDA_MIN"]
pdata["max_demand_param"] = factory_dict["DEMANDA_MAX"]
pdata["min_rsp_param"] = factory_dict["RSP_MIN"]
pdata["max_rsp_param"] = factory_dict["RSP_MAX"]

route_dict = pdata["ROTA"].set_index(["ORIGEM", "TRANSPORTADOR", "DESTINO"]).to_dict()
pdata["cycle_param"] = route_dict["TEMPO_CICLO"]
route_dict_v2 = pdata["ROTA"].set_index(["TRANSPORTADOR"]).to_dict()
pdata["fitting_box_param"] = route_dict_v2["CAIXA_CARGA"]

In [2201]:
pdata["route_possibilities_set"] = route_dict["TEMPO_CICLO"].keys()

### Create Model

In [2202]:
model = pyo.ConcreteModel()

### Sets

In [2203]:
model.period = pyo.Set(initialize=pdata["period_set"])
model.supplier = pyo.Set(initialize=pdata["logistics_set"])
model.up = pyo.Set(initialize=pdata["up_set"])
model.factory = pyo.Set(initialize=pdata["factory_set"])
model.route_possibilities = pyo.Set(initialize=pdata["route_possibilities_set"])

### Parameters

In [2204]:
model.min_trucks = pyo.Param(model.supplier, initialize=pdata["min_trucks_param"])
model.max_trucks = pyo.Param(model.supplier, initialize=pdata["max_trucks_param"])

model.grua = pyo.Param(model.supplier, initialize=pdata["grua_param"])

model.density = pyo.Param(model.up, initialize=pdata["density_param"])
model.volume = pyo.Param(model.up, initialize=pdata["volume_param"])
model.rsp = pyo.Param(model.up, initialize=pdata["rsp_param"])

model.min_demand = pyo.Param(model.factory, model.period, initialize=pdata["min_demand_param"])
model.max_demand = pyo.Param(model.factory, model.period, initialize=pdata["max_demand_param"])
model.min_rsp = pyo.Param(model.factory, model.period, initialize=pdata["min_rsp_param"])
model.max_rsp = pyo.Param(model.factory, model.period, initialize=pdata["max_rsp_param"])

model.cycle = pyo.Param(model.route_possibilities, initialize=pdata["cycle_param"])
model.fitting_box = pyo.Param(model.supplier, initialize=pdata["fitting_box_param"])

### Variables

In [2205]:
model.amount_of_trucks = pyo.Var(model.up, model.supplier, model.factory, model.period, domain=pyo.NonNegativeReals)
model.amount_delivery = pyo.Var(model.up, model.supplier, model.factory, model.period, domain=pyo.NonNegativeReals)
model.uniqueness = pyo.Var(model.up, model.supplier, model.factory, model.period, domain=pyo.Binary)
model.addressed = pyo.Var(model.up, model.supplier, model.factory, model.period, domain=pyo.Binary)

model.volume_art = pyo.Var(model.up)

### Constraint

In [2206]:
def raw_material_minimum_delivery(model, f, p):

    eq = sum(
        model.amount_delivery[u, s, f, p]
        for u in model.up
        for s in model.supplier
        if (u, s, f) in model.route_possibilities
     ) >= model.min_demand[f, p]
    
    return eq

model.raw_material_minimum_delivery = pyo.Constraint(
    model.factory, model.period, rule=raw_material_minimum_delivery
)

In [2207]:
def raw_material_maximum_delivery(model, f, p):
    eq = sum(
        model.amount_delivery[u, s, f, p]
        for u in model.up
        for s in model.supplier
        if (u, s, f) in model.route_possibilities
     ) <= model.max_demand[f, p]
    
    return eq

model.raw_material_maximum_delivery = pyo.Constraint(
    model.factory, model.period, rule=raw_material_maximum_delivery
)

In [2208]:
def attend_maximum_rsp(model, f, p):
    delivery_qty_rsp = sum(
        model.amount_delivery[u, s, f, p] * model.rsp[u]
        for u in model.up
        for s in model.supplier
    )
    delivery_qty_rsp_max = sum(
        model.amount_delivery[u, s, f, p] * model.max_rsp[f, p]
        for u in model.up
        for s in model.supplier
    )

    eq = delivery_qty_rsp <= delivery_qty_rsp_max
    return eq

model.attend_maximum_rsp = pyo.Constraint(
    model.factory, model.period, rule=attend_maximum_rsp
)

In [2209]:
def attend_minimum_rsp(model, f, p):
    delivery_qty_rsp = sum(
        model.amount_delivery[u, s, f, p] * model.rsp[u]
        for u in model.up
        for s in model.supplier
    )
    delivery_qty_rsp_min = sum(
        model.amount_delivery[u, s, f, p] * model.min_rsp[f, p]
        for u in model.up
        for s in model.supplier
    )

    eq = delivery_qty_rsp >= delivery_qty_rsp_min
    return eq

model.attend_minimum_rsp = pyo.Constraint(
    model.factory, model.period, rule=attend_minimum_rsp
)

In [2210]:
def attend_total_volume_of_up(model, u):
    total_volume_up = sum(
        model.amount_delivery[u, s, f, p]
        for s in model.supplier
        for f in model.factory
        for p in model.period
    )

    eq = total_volume_up == model.volume[u]# * addressing
    print(eq)
    return eq

model.attend_total_volume_of_up = pyo.Constraint(
    model.up, rule=attend_total_volume_of_up
)

amount_delivery[S6C298,Pastori,LIM,1] + amount_delivery[S6C298,Pastori,LIM,2] + amount_delivery[S6C298,Pastori,LIM,3] + amount_delivery[S6C298,Pastori,LIM,4] + amount_delivery[S6C298,Pastori,LIM,5] + amount_delivery[S6C298,Pastori,LIM,6] + amount_delivery[S6C298,Pastori,LIM,7] + amount_delivery[S6C298,Pastori,LIM,8] + amount_delivery[S6C298,Pastori,LIM,9] + amount_delivery[S6C298,Pastori,LIM,10] + amount_delivery[S6C298,Pastori,LIM,11] + amount_delivery[S6C298,Pastori,LIM,12] + amount_delivery[S6C298,Pastori,LIM,13] + amount_delivery[S6C298,Pastori,LIM,14] + amount_delivery[S6C298,Pastori,LIM,15] + amount_delivery[S6C298,Pastori,LIM,16] + amount_delivery[S6C298,Pastori,LIM,17] + amount_delivery[S6C298,Pastori,LIM,18] + amount_delivery[S6C298,Pastori,LIM,19] + amount_delivery[S6C298,Pastori,LIM,20] + amount_delivery[S6C298,Pastori,LIM,21] + amount_delivery[S6C298,Pastori,LIM,22] + amount_delivery[S6C298,Pastori,LIM,23] + amount_delivery[S6C298,Pastori,LIM,24] + amount_delivery[S6C298,Pa

In [2211]:
def transport_capacity(model, u, f, p):
    total_volume = sum(
        model.amount_delivery[u, s, f, p]
        for s in model.supplier
    )

    qty_trucks= sum(
        model.amount_of_trucks[u, s, f, p]
        for s in model.supplier
    )

    fitting_box = sum(
        model.fitting_box[s]
        for s in model.supplier
    )

    cycle = sum(
        model.cycle[u, s, f]
        for s in model.supplier
        if (u, s, f) in model.route_possibilities
    )

    eq = total_volume <= qty_trucks * fitting_box * 8 / (cycle * 2)
    print(eq)
    return eq

model.transport_capacity = pyo.Constraint(
    model.up, model.factory, model.period, rule=transport_capacity
)

amount_delivery[S6C298,Pastori,LIM,1] + amount_delivery[S6C298,Tover,LIM,1] + amount_delivery[S6C298,Rampazo,LIM,1]  <=  (amount_of_trucks[S6C298,Pastori,LIM,1] + amount_of_trucks[S6C298,Tover,LIM,1] + amount_of_trucks[S6C298,Rampazo,LIM,1])*198*8/4.2
amount_delivery[S6C298,Pastori,LIM,2] + amount_delivery[S6C298,Tover,LIM,2] + amount_delivery[S6C298,Rampazo,LIM,2]  <=  (amount_of_trucks[S6C298,Pastori,LIM,2] + amount_of_trucks[S6C298,Tover,LIM,2] + amount_of_trucks[S6C298,Rampazo,LIM,2])*198*8/4.2
amount_delivery[S6C298,Pastori,LIM,3] + amount_delivery[S6C298,Tover,LIM,3] + amount_delivery[S6C298,Rampazo,LIM,3]  <=  (amount_of_trucks[S6C298,Pastori,LIM,3] + amount_of_trucks[S6C298,Tover,LIM,3] + amount_of_trucks[S6C298,Rampazo,LIM,3])*198*8/4.2
amount_delivery[S6C298,Pastori,LIM,4] + amount_delivery[S6C298,Tover,LIM,4] + amount_delivery[S6C298,Rampazo,LIM,4]  <=  (amount_of_trucks[S6C298,Pastori,LIM,4] + amount_of_trucks[S6C298,Tover,LIM,4] + amount_of_trucks[S6C298,Rampazo,LIM,4])*19

In [2212]:
model.transport_capacity.display()

transport_capacity : Size=806
    Key                   : Lower : Body : Upper
     ('S3AX01', 'LIM', 1) :  None : None :   0.0
     ('S3AX01', 'LIM', 2) :  None : None :   0.0
     ('S3AX01', 'LIM', 3) :  None : None :   0.0
     ('S3AX01', 'LIM', 4) :  None : None :   0.0
     ('S3AX01', 'LIM', 5) :  None : None :   0.0
     ('S3AX01', 'LIM', 6) :  None : None :   0.0
     ('S3AX01', 'LIM', 7) :  None : None :   0.0
     ('S3AX01', 'LIM', 8) :  None : None :   0.0
     ('S3AX01', 'LIM', 9) :  None : None :   0.0
    ('S3AX01', 'LIM', 10) :  None : None :   0.0
    ('S3AX01', 'LIM', 11) :  None : None :   0.0
    ('S3AX01', 'LIM', 12) :  None : None :   0.0
    ('S3AX01', 'LIM', 13) :  None : None :   0.0
    ('S3AX01', 'LIM', 14) :  None : None :   0.0
    ('S3AX01', 'LIM', 15) :  None : None :   0.0
    ('S3AX01', 'LIM', 16) :  None : None :   0.0
    ('S3AX01', 'LIM', 17) :  None : None :   0.0
    ('S3AX01', 'LIM', 18) :  None : None :   0.0
    ('S3AX01', 'LIM', 19) :  None : Non

In [2213]:
def maximum_amount_of_trucks(model, s, p):
    eq = sum(
    model.amount_of_trucks[u, s, f, p] 
    for u in model.up
    for f in model.factory
    ) <= model.max_trucks[s]
    return eq

model.maximum_amount_of_trucks = pyo.Constraint(
    model.supplier, model.period, rule=maximum_amount_of_trucks
)

In [2214]:
def minimum_amount_of_trucks(model, s, p):
    eq = sum(
    model.amount_of_trucks[u, s, f, p] 
    for u in model.up
    for f in model.factory
    ) >= model.min_trucks[s]
    return eq

model.minimum_amount_of_trucks = pyo.Constraint(
    model.supplier, model.period, rule=minimum_amount_of_trucks
)

In [2215]:
def addressing_supplier(model, u, s, f, p):
    M = 210000
    eq = model.amount_of_trucks[u, s, f, p] <= model.addressed[u, s, f, p] * M

    return eq

model.addressing_supplier = pyo.Constraint(
    model.up, model.supplier, model.factory, model.period, rule=addressing_supplier
)

In [2216]:
def activate_addressing_supplier(model, u, s, f, p):
    M = 210000
    eq = model.addressed[u, s, f, p] <= model.amount_delivery[u, s, f, p] * M

    return eq

model.activate_addressing_supplier = pyo.Constraint(
    model.up, model.supplier, model.factory, model.period, rule=activate_addressing_supplier
)

In [2217]:
# def transport_limit_per_day(model, u, s, p):
#     qty_delivered = sum(
#         model.amount_delivery[u, s, f, p]
#         for u in model.up
#         for s in model.supplier
#         for f in model.factory
#     )

#     qty_of_trucks = sum(
#         model.amount_of_trucks[u, s, f, p]
#         for f in model.factory
#     )

#     cycles = sum(
#         model.cycle[u, s, f]
#         for f in model.factory
#         if (u, s, f) in model.route_possibilities
#     )

#     eq = qty_delivered <= qty_of_trucks * model.fitting_box[s] * cycles
#     print(eq)

#     return eq

# model.transport_limit_per_day = pyo.Constraint(
#     model.up, model.supplier, model.period, rule=transport_limit_per_day
# )
    

### Objetive

In [2218]:
def objetive_function(model):
    eq = sum(
        model.amount_delivery[u, s, f, p]
        for u in model.up
        for s in model.supplier
        for f in model.factory
        for p in model.period
        if (u, s, f) in model.amount_delivery
    )

    # eq2 =  sum(
    #     model.volume_art[u]
    #     for u in model.up
    # )

    return eq #+ eq2

model.objetive_function = pyo.Objective(
    rule=objetive_function, sense=pyo.minimize
)

### Solve model

In [2219]:
model.write("optimization_test_suzano.lp", io_options={"symbolic_solver_labels":True})

solver = pyo.SolverFactory("appsi_highs")
solver.solve(model)

{'Problem': [{'Lower bound': 0.0, 'Upper bound': 0.0, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Termination message': 'TerminationCondition.optimal'}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [2220]:
sum(model.min_demand[:, :])

198400

In [2221]:
model.amount_of_trucks.display()

amount_of_trucks : Size=2418, Index=up*supplier*factory*period
    Key                              : Lower : Value                  : Upper : Fixed : Stale : Domain
     ('S3AX01', 'Pastori', 'LIM', 1) :     0 :                    0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 2) :     0 :                    0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 3) :     0 : 2.1645021645021644e-08 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 4) :     0 :                    0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 5) :     0 :                    0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 6) :     0 :                    0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 7) :     0 : 2.1645021645021644e-08 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 8) :     

In [2222]:
model.amount_delivery.display()

amount_delivery : Size=2418, Index=up*supplier*factory*period
    Key                              : Lower : Value                  : Upper : Fixed : Stale : Domain
     ('S3AX01', 'Pastori', 'LIM', 1) :     0 :                    0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 2) :     0 :                    0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 3) :     0 : 4.7619047619047615e-06 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 4) :     0 :                    0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 5) :     0 :                    0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 6) :     0 :                    0.0 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 7) :     0 : 4.7619047619047615e-06 :  None : False : False : NonNegativeReals
     ('S3AX01', 'Pastori', 'LIM', 8) :     0

In [2223]:
sum(model.amount_delivery[:, :, :, :]())

200723.99999999997

In [2224]:
model.maximum_amount_of_trucks.display()

maximum_amount_of_trucks : Size=93
    Key             : Lower : Body               : Upper
     ('Pastori', 1) :  None :               27.0 :  27.0
     ('Pastori', 2) :  None :               27.0 :  27.0
     ('Pastori', 3) :  None :               27.0 :  27.0
     ('Pastori', 4) :  None :               27.0 :  27.0
     ('Pastori', 5) :  None :               27.0 :  27.0
     ('Pastori', 6) :  None :               27.0 :  27.0
     ('Pastori', 7) :  None :               27.0 :  27.0
     ('Pastori', 8) :  None :               27.0 :  27.0
     ('Pastori', 9) :  None :               27.0 :  27.0
    ('Pastori', 10) :  None :               27.0 :  27.0
    ('Pastori', 11) :  None :               27.0 :  27.0
    ('Pastori', 12) :  None :               27.0 :  27.0
    ('Pastori', 13) :  None :               27.0 :  27.0
    ('Pastori', 14) :  None :               27.0 :  27.0
    ('Pastori', 15) :  None :               27.0 :  27.0
    ('Pastori', 16) :  None :               27.0 :  2

In [2225]:
model.minimum_amount_of_trucks.display()

minimum_amount_of_trucks : Size=93
    Key             : Lower : Body               : Upper
     ('Pastori', 1) :  25.0 :               27.0 :  None
     ('Pastori', 2) :  25.0 :               27.0 :  None
     ('Pastori', 3) :  25.0 :               27.0 :  None
     ('Pastori', 4) :  25.0 :               27.0 :  None
     ('Pastori', 5) :  25.0 :               27.0 :  None
     ('Pastori', 6) :  25.0 :               27.0 :  None
     ('Pastori', 7) :  25.0 :               27.0 :  None
     ('Pastori', 8) :  25.0 :               27.0 :  None
     ('Pastori', 9) :  25.0 :               27.0 :  None
    ('Pastori', 10) :  25.0 :               27.0 :  None
    ('Pastori', 11) :  25.0 :               27.0 :  None
    ('Pastori', 12) :  25.0 :               27.0 :  None
    ('Pastori', 13) :  25.0 :               27.0 :  None
    ('Pastori', 14) :  25.0 :               27.0 :  None
    ('Pastori', 15) :  25.0 :               27.0 :  None
    ('Pastori', 16) :  25.0 :               27.0 :  N

In [2226]:
model.addressed.display()

addressed : Size=2418, Index=up*supplier*factory*period
    Key                              : Lower : Value : Upper : Fixed : Stale : Domain
     ('S3AX01', 'Pastori', 'LIM', 1) :     0 :   0.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 2) :     0 :   0.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 3) :     0 :   1.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 4) :     0 :   0.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 5) :     0 :   0.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 6) :     0 :   0.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 7) :     0 :   1.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 8) :     0 :   0.0 :     1 : False : False : Binary
     ('S3AX01', 'Pastori', 'LIM', 9) :     0 :   0.0 :     1 : False : False : Binary
    ('S3AX01', 'Pastori', 'LIM', 10) :     0 :   1.0 :     1 : False : False : Binar